In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
pd.set_option('display.float_format', '{:.2f}'.format)

In [4]:
# without csv
import requests
from bs4 import BeautifulSoup

result = requests.get("https://gppbgovph.com/awardedcontracts.php")
src = str(result.content)

end = src.find("editoptions\":{\"value")
start = src.find("All;")
final_legend = src[start+4:end-4]

final_legend = final_legend.split(";")
legend_dict = {}
for x in final_legend:
    new_legend = x.split(":")
    legend_dict[int(new_legend[0])] = new_legend[1]

In [5]:
df = pd.read_excel("Grid_AwardedContracts_2020-08-03.xlsx")

In [6]:
remove_these_columns = ["Notice Of Award","Contract/PO","Annual Procurement Plan"]
df.drop(columns=remove_these_columns,inplace=True)

In [7]:
df.head()

,Project Name,Approved Budget,Start Date,End Date,Name Of Awarded Company,Amount Awarded,Date Of Award,Acceptance Date,Procuring Entity
0,Human Grade Disinfectant Solution and Face Mas...,108864.00,2020-04-08,2020-04-18,Bauertek Corporation,108864.00,2020-04-02,2020-04-08,2
1,Procurement of relief goods,686000.00,2020-04-16,2020-04-23,Tina's Dried Fish and Egg Dealer,632800.00,2020-04-16,2020-04-16,12
2,Procurement of relief goods,686000.00,2020-04-16,2020-04-23,Tina's Dried Fish and Eggs,632800.00,2020-04-15,2020-04-15,12
3,Procurement of Relief Goods,1577000.00,2020-04-17,2020-04-24,Green Season Rice Mill & Grain Center,1535500.00,2020-04-15,2020-04-15,12
4,Procurement of Various Medical Supplies for It...,64900000.00,2020-03-24,2020-05-22,"Rebmann, Inc.",64790000.00,2020-03-16,2020-03-18,42


In [8]:
df['Procuring Entity'] = df['Procuring Entity'].replace(legend_dict)

In [9]:
df_sorted = df.copy().sort_values(by="Amount Awarded",ascending=False)
df_sorted.reset_index(drop=True,inplace=True)

In [10]:
df_sorted.head()

,Project Name,Approved Budget,Start Date,End Date,Name Of Awarded Company,Amount Awarded,Date Of Award,Acceptance Date,Procuring Entity
0,Supply and Delivery of Personal Protective Equ...,3846000000.00,2020-05-08,2020-06-07,Pharmally Pharmaceutical Corporation,3820000000.00,2020-05-06,2020-05-06,PROCUREMENT SERVICE
1,Supply and Delivery of Testing Kits for COVID ...,2877300000.00,2020-06-09,2020-06-24,Pharmally Pharmaceutical Corporation,2877300000.00,2020-06-08,2020-06-08,Procurement Service - DBM
2,SUPPLY AND DELIVERY OF PERSONAL PROTECTIVE EQU...,1923000000.00,2020-05-08,2020-06-07,XUZHOU CONSTRUCTION MACHINERY GROUP IMP. AND E...,1898000000.00,2020-05-07,2020-05-07,PROCUREMENT SERVICE
3,Supply and Delivery of Personal Protective Equ...,790000000.00,2020-06-29,2020-08-13,Ferjan Healthlink Philippines Inc.,727500000.00,2020-06-19,2020-06-19,PROCUREMENT SERVICE
4,Supply and Delivery of Testing Kits for COVID ...,760000000.00,2020-04-24,2020-07-31,Pharmally Pharmaceutical Corporation,688000000.00,2020-04-23,2020-04-23,Procurement Service - DBM


In [11]:
df_sorted["Money Ratio"] = df_sorted["Amount Awarded"].div(df_sorted["Approved Budget"])

In [12]:
total_amount_awarded = df_sorted["Amount Awarded"].sum(axis=0)
total_amount_awarded = total_amount_awarded.round(2)
total_amount_awarded

35955162449.02

In [13]:
df_cleaned = df_sorted.copy()
df_cleaned["Category"] = "Not classified"

#### Do not run the cell below this (manual input cell below)

In [ ]:
#Manual renaming per row, manually noted "keywords" for automatic sorting later on

money = 0
indices = []
for index,row in df_cleaned.iterrows():
    
    if money/total_amount_awarded >= 0.75:
        break
    else:
        indices.append(index)
    
        if index%20 == 0:
            print("\n",index,"\n")

        df_cleaned.loc[index,"Category"] = input(df_cleaned.iloc[index,0]+" :  ")
        money += row[5]

#### Do not run the cell above this (manual input cell below)

In [56]:
money = 0
indices = []
num_of_rows = len(df_cleaned)
while True:
    try:
        cutoff_float = float(input("Enter a number for the cutoff point: "))

        while cutoff_float > 1 or cutoff_float < 0:
            try:
                print("Invalid Input.")
                cutoff_float = float(input("Enter a number for the cutoff point: "))

            except:
                pass

        for index,row in df_cleaned.iterrows():
            if money/total_amount_awarded >= cutoff_float:
                break
            else:
                indices.append(index)
                money += row[5]

        print("The top",indices[-1]+1,"rows (out of {num}) comprise at least {cutoff:.2f}% of the Total Amount Awarded".format(num = num_of_rows, cutoff = cutoff_float*100))
        break
    except:
        print("Invalid Input.")



Enter a number for the cutoff point: 0.75
The top 422 rows (out of 11697) comprise at least 75.00% of the Total Amount Awarded


In [59]:
indices = []
num_of_rows = len(df_cleaned)
money = 0
while True:
    try:
        cutoff_flat = float(input('Values greater than or equal to this amount is considered "big": '))
        
        for index,row in df_cleaned.iterrows():
            if row[5] < cutoff_flat:
                break
            else:
                indices.append(index)
                money += row[5]
        
        rows = indices[-1]+1
        print("\nThe top",rows,'rows (out of {num}) are considered "big" bids ({percentage:.2f}% of all rows).'.format(num=num_of_rows,percentage = rows/num_of_rows*100))
        print('Together, all these "big" bids equal to {percentage:.2f}% of the Total Amount Awarded'.format(percentage=money/total_amount_awarded*100))
        print('\nThe remaining {diff} rows are all considered "small" bids ({percentage:.2f}% of all rows).'.format(diff = num_of_rows - rows, percentage = (1-(rows/num_of_rows))*100))
        break
    except:
        print("Invalid Input.")

Values greater than or equal to this amount is considered "big": 10000000

The top 440 rows (out of 11697) are considered "big" bids (3.76% of all rows).
Together, all these "big" bids equal to 75.53% of the Total Amount Awarded

The remaining 11257 rows are all considered "small" bids (96.24% of all rows).


In [25]:
#index 0-421 comprise >= 0.75 of the total Amount Awarded
indices[-1]

11696

# Note to self:  

### Keywords:  
Medical = [Personal Protective Equipment, PPE, Department of Health, DOH, Testing Kit, Test Kit, N95, Surgical Mask, Pipette, Hygiene Kit, Gown, Goggles, Alcohol, Gloves, Coverall, ventilator, Medical, Hospital, Health, RT-PCR, testing, medicin, detection, vitamin, mask, laboratory, lab, PCR, extraction, kit, CT Scan]
  
Food = [Rice, Corned Beef, Beef Loaf, Relief Good, Seed, NSIC, Food, Water, grocery, Sardines, milk]  

### Not sure if correct:


CT Scan Machine (categorized as Medical, index is somewhere in (120-140))  
patient vehicle (categorized as Others, index is somewhere in (220-240))  

Construction of Hospital-Based Health Isolation Facilities for COVID-19 Patients and Health Care Professionals, EVRMC Cabalawan, Tacloban City :  Medical (280-300)

FURNISHING LABOR AND MATERIALS FOR THE CONSTRUCTION OF BULACAN MOLECULAR DIAGNOSTIC LABORATORY AT BULACAN MEDICAL CENTER, CAPITOL COMPOUND, CITY OF MALOLOS, BULACAN :  Others (360-380)

In [ ]:
#Weird Observation regarding Money Ratio... disregard na lang if ever???
df_cleaned[df_cleaned["Money Ratio"]>1].head()

# 4. Categorizing by money procured by the procuring entity

In [26]:
df_Money_by_ProcuringEntity = df_cleaned.copy()

removelist = ['Project Name', 'Approved Budget', 'Start Date', 'End Date',
       'Name Of Awarded Company', 'Date Of Award',
       'Acceptance Date', 'Money Ratio', 'Category']

df_Money_by_ProcuringEntity.drop(columns = removelist,inplace=True)

df_Money_by_ProcuringEntity = df_Money_by_ProcuringEntity[["Procuring Entity","Amount Awarded"]]



df_sorted_Money_by_ProcuringEntity = df_Money_by_ProcuringEntity.groupby(["Procuring Entity"]).sum().copy()

df_sorted_Money_by_ProcuringEntity.sort_values(by="Amount Awarded",ascending=False,inplace=True)

df_sorted_Money_by_ProcuringEntity.reset_index(inplace=True)

total = total_amount_awarded.copy()

df_sorted_Money_by_ProcuringEntity["Ratio from Total Money"] = df_sorted_Money_by_ProcuringEntity["Amount Awarded"].divide(total)

df_sorted_Money_by_ProcuringEntity.head()


,Procuring Entity,Amount Awarded,Ratio from Total Money
0,PROCUREMENT SERVICE,10089578194.30,0.28
1,Procurement Service - DBM,4941726625.00,0.14
2,City Government of Taguig,2013432750.54,0.06
3,Department of Health,990810937.50,0.03
4,Office of the Chie Minister-Bangsamoro Autonom...,895257427.30,0.02


# Disregard below... safety precautions when I restarted everything lang... 

## Delete all the ones below if there is automatic categorization na :))  ~Aids

In [18]:
df_category = pd.read_csv("Manually_Categorized.csv")
df_category = df_category["Category"]
df_category

0               Medical
1               Medical
2               Medical
3               Medical
4               Medical
              ...      
11692    Not classified
11693    Not classified
11694    Not classified
11695    Not classified
11696    Not classified
Name: Category, Length: 11697, dtype: object

In [19]:
df_cleaned["Category"] = df_category